In [1]:
from __future__ import division
from hmmlearn.hmm import MultinomialHMM
import pandas as pd
import numpy as np

import json

from  Preprocesscopy import Preprocesscopy

from sklearn.cross_validation import train_test_split

In [2]:
file_path = '/home/rui/Documents/MLProject/data/modified-crab/trips-in-sanfran.csv'
city = 'san_francisco'
tile_size = 0.360
la_size = 0.003234
long_size = 0.004049
start_id = 0
freq_threshold = 0

In [3]:
data = Preprocesscopy(file_path, city, tile_size, la_size, long_size, freq_threshold)

In [4]:
data.read_dataset()
print "done read dataset"
data.split_train_test()
print "done split dataset"
data.find_significant_region(start_id)
print "done find significant region"
data.filter_low_frequency_trajectory()
print "done filter low frequency traj"
data.refind_significant_region(start_id)
print "done refind significant region"
data.convert_train_traj_to_region_id()
print "done convert train traj to region id"
data.convert_test_traj_to_region_id()
print "done convert test traj to region id"

done read dataset
done split dataset
done find significant region
done filter low frequency traj
done refind significant region
done convert train traj to region id
done convert test traj to region id


In [5]:
taxi_train = data.taxi_train
taxi_test = data.taxi_test

In [6]:
# define a function to convert trajactro to a numpy array
def trace2train_format(training_data):
    train_locations = []
    train_lengths = []
    for row_index, row in training_data.iterrows():
        single_trip = row[3]
        trip_length = len(single_trip)
        
        train_lengths.append(trip_length)
        
        for i in range(trip_length):
            train_locations.append(single_trip[i])

    train_locations = np.array(train_locations)
    train_lengths = np.array(train_lengths)
    return train_locations, train_lengths

In [7]:
train_locations, train_lengths = trace2train_format(taxi_train)

In [ ]:
hmm1 = MultinomialHMM(n_components=16, n_iter=500, verbose=True)
train_locations = np.column_stack([train_locations])
hmm1.fit(train_locations, train_lengths)

In [11]:
 from sklearn.externals import joblib
joblib.dump(hmm1, "hiddenmarkovmodel360-12.pkl")

['hiddenmarkovmodel360-12.pkl',
 'hiddenmarkovmodel360-12.pkl_01.npy',
 'hiddenmarkovmodel360-12.pkl_02.npy',
 'hiddenmarkovmodel360-12.pkl_03.npy',
 'hiddenmarkovmodel360-12.pkl_04.npy']

In [ ]:
joblib.load("hiddenmarkovmodel.pkl")  

360m  8 hidden states
hiddenmarkovmodel360-4: 468    -5744872.8860          +0.0087


360m  8 hidden states
hiddenmarkovmodel360-8: 500    -5277765.4383          +3.8056

360m  12 hidden states
hiddenmarkovmodel360-12: 500    -4972390.1166          +0.2825

360m  16 hidden states
hiddenmarkovmodel360-16: 500    -4795193.7351          +2.6486

360m  20 hidden states
hiddenmarkovmodel360-20: 500    -4683460.6453          +1.8457

360m  24 hidden states
hiddenmarkovmodel360-24: 500    -4558716.5066          +0.6078

# test

In [ ]:
def predict(x):
    predictions = []
    # parameters of hidden markov model
    transmat = transmat_
    emissionprob = emissionprob_
    startprob = np.column_stack([startprob_])

    # alpha recursion
    T = len(x)
    H = n_components

    alpha = np.zeros((H, T))
    alpha = np.asmatrix(alpha)
    z = np.zeros(T)  # local normalisaton factors

    phgh = transmat.T

    alpha[:, 0] = emissionprob[:, x[0]] * startprob

    z[0] = alpha[:, 0].sum()

    alpha[:, 0] = alpha[:, 0] / z[0]

    for t in range(1, T):
        predictor = phgh.dot(alpha[:, t-1])
        alpha[:, t] = emissionprob[:, x[t]] * np.array(predictor)
        z[t] = alpha[:, t].sum()
        alpha[:, t] = alpha[:, t] / z[t]
        next_hidden = phgh.dot(alpha[:, 0])
        next_observation = emissionprob.T.dot(next_hidden)
        next_prediction = np.where(next_observation == np.max(next_observation))
        predictions.append(next_prediction[0][0])

    return predictions